In [1]:
from configs import train_config
config = train_config.Config()
import sys
import os
sys.path.append(config.coco_api)
from pycocotools.coco import COCO
import cv2
from random import shuffle
import tensorflow as tf

train_annotation_path = os.path.join(config.dataset_dir, 'annotations', 'CNSI-SED-TRAIN.json')
coco = COCO(train_annotation_path)
ids = coco.getImgIds()
#ids = shuffle(coco.getImgIds())

print(ids)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '/home/docker_sharing_folder/sed-datasets/annotations/CNSI-SED-TRAIN.json'

In [ ]:
def set_id_to_label(label_set):
    id_to_label = {}
    for idx, label in enumerate(label_set):
        id_to_label[idx+1] = label
    return id_to_label

In [ ]:
image_dir = '/home/dan/1T/SED-project/sed-datasets/SHWD/VOC2028/JPEGImages'
input_shape = (512, 512)
id_to_label = set_id_to_label(['head', 'helmet'])
cat_ids = coco.getCatIds(id_to_label.values())

In [2]:
print(id_to_label)
print(cat_ids)

NameError: name 'id_to_label' is not defined

In [ ]:
def _get_image(image_id, coco):
    image_info = coco.loadImgs(image_id)[0]
    filename = image_info['file_name']
    original_size = (int(image_info['height']), int(image_info['width']))
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)
    if image is None:
        filename = os.path.splitext(filename)[0] + '.JPG'
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError("Can't not find correct filepath", filename)
        print(filename)
    image = image[:,:,::-1]
    image = cv2.resize(image, input_shape)
    image = (image / 127.0) - 1.0
    return filename, image, original_size

In [ ]:
def _get_annotation(image_id, cat_ids, original_size, coco):
    ann_ids = coco.getAnnIds(imgIds=image_id, catIds=cat_ids, iscrowd=None)
    boxes = []
    (image_height, image_width) = original_size[0], original_size[1]
    for ann_id in ann_ids:
        x, y, w, h = coco.loadAnns(ann_id)[0]['bbox']
        xmin = x / image_width
        ymin = y / image_height
        xmax = (x+w)/image_width
        ymax = (y+h)/image_height
        box = [xmin, ymin, xmax, ymax]
        boxes.append(box)
    labels = [ coco.loadAnns(ann_id)[0]['category_id'] for ann_id in ann_ids]
    return boxes, labels

In [ ]:
import numpy as np
from utils import anchor, box_utils
from configs import train_config
config = train_config.Config()

In [ ]:
filename, image, original_size = _get_image([5], coco)

In [ ]:
boxes, labels = _get_annotation([5], cat_ids, original_size, coco)

In [ ]:
boxes

In [ ]:
default_boxes = anchor.generate_default_boxes(config)
transformed_default_boxes = box_utils.transform_center_to_corner(default_boxes)

In [ ]:
iou = box_utils.compute_iou(transformed_default_boxes, boxes)

In [ ]:
iou

In [ ]:
np.sort(iou[iou>0])

In [ ]:
print(default_boxes.shape)

In [ ]:
transformed_default_boxes = box_utils.transform_center_to_corner(default_boxes)

In [ ]:
for _id in ids[:100]:
    filename, image, original_size  = _get_image([_id], coco)
    boxes, labels = _get_annotation(_id, cat_ids, original_size, coco)
    print(_id, " : annotation 갯수", len(labels))
    IOU = box_utils.compute_iou(transformed_default_boxes, boxes)
    print("0.5 IOU 이상의 anchorbox 갯수", len(IOU[IOU>0.5]))

In [ ]:
for _id in ids[:100]:
    filename, image, original_size  = _get_image([_id], coco)
    boxes, labels = _get_annotation(_id, cat_ids, original_size, coco)
    print(_id, " : annotation 갯수", len(labels))
    IOU = box_utils.compute_iou(transformed_default_boxes, boxes)
    print("0.5 IOU 이상의 anchorbox 갯수", len(IOU[IOU>0.5]))
    
    gt_confs, gt_locs = box_utils.compute_target(default_boxes, boxes, labels, iou_threshold=0.5)
    print("gt_config and gt_locs", tf.sort(gt_confs), gt_locs)
    print("--------------------------------------------------------------------------------")